# raw, strength and specialization

In [17]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly

In [18]:
df_sdg = pd.read_pickle("../data/dataframes/output/all_sdg_fixed_dst.pkl")
df_digital = pd.read_pickle("../data/dataframes/DT/dt_updated.pkl")
eu = pd.read_excel("../data/countries_eu.xlsx", sheet_name='EU')['Country'].to_list()
# Filtering for Journal only
df_sdg = df_sdg[df_sdg.PT == 'J']
df_digital = df_digital[df_digital.PT == 'J']
df_sdg_dt = df_sdg[df_sdg['DST']]

idx_sdg = {name: i for i, name in enumerate(list(df_sdg), start=0)}
idx_dt = {name: i for i, name in enumerate(list(df_digital), start=0)}
idx_sdg_dt = {name: i for i, name in enumerate(list(df_sdg_dt), start=0)}

In [19]:
gdp = pd.read_excel("../data/pop-gdp.xlsx", sheet_name="GDP", index_col="TIME").rename(columns={"2020-Q2":"GDP"}).loc[:, "GDP"].rename_axis("Country", axis=0)
gdp

Country
NaN                                                NaN
EU                                           3092983.9
European Union - 28 countries (2013-2020)            :
Euro area - 19 countries  (from 2015)        2631060.9
Belgium                                         105644
                                               ...    
Palestine                                            :
Saudi Arabia                                         :
Australia                                     280349.7
New Zealand                                      40983
Samoa                                                :
Name: GDP, Length: 102, dtype: object

In [20]:
eu_countries = pd.read_excel("../data/countries_eu.xlsx", sheet_name='EU').Country.to_list()
other_countries = pd.read_excel("../data/countries_eu.xlsx", sheet_name='rest').Country.to_list()

gdppop = pd.read_excel("../data/gdp_avgpop.xlsx", index_col=0).replace(":", np.NAN).fillna(0)
gdppop

,avg_pop,gdp
Albania,2.875263e+06,3348.7
Argentina,4.333679e+07,0.0
Armenia,2.924618e+06,0.0
Australia,2.400823e+07,324962.5
Austria,8.665144e+06,92140.6
...,...,...
Ukraine,4.498976e+07,0.0
Uruguay,3.419979e+06,0.0
United States,3.213942e+08,4518929.3
Samoa,1.933848e+05,0.0


In [21]:
"""
# Digital

dic_countries = {country: [0, 0, 0]  for country in gdppop.index}
for row in df_digital.itertuples(index=False, name=None):
    lst_actors = row[idx_dt['CN']].split(", ")
    year = row[idx_dt['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries.keys():
            # Increment for digital
            dic_countries[actor][0] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][0] += 1 / size_actors

# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    lst_actors = row[idx_sdg['CN']].split(", ")
    year = row[idx_sdg['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries.keys():
            # Increment for SDG
            dic_countries[actor][1] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][1] += 1 / size_actors
            if row[-1]:
                # Increment for SDG-DT
                dic_countries[actor][2] += 1 / size_actors
                if actor in eu_countries:
                    dic_countries['EU'][2] += 1 / size_actors
"""
from collections import Counter
dic_countries_frac = {country:Counter() for country in gdppop.index}
# Digital
for row in df_digital.itertuples(index=False, name=None):
    cn = row[idx_dt['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries_frac.keys():
            # Increment for digital
            dic_countries_frac[actor]['DT'] += 1/size_actors

            if actor in eu:
                dic_countries_frac['EU']['DT'] += 1/size_actors

# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[idx_sdg['CN']]
    lst_actors = cn.split(", ")
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries_frac.keys():
            # Increment for SDG
            dic_countries_frac[actor]['SDG'] += 1/size_actors

            if actor in eu:
                dic_countries_frac['EU']['SDG'] += 1/size_actors

            if row[-1]:
                # Increment for SDG-DT
                dic_countries_frac[actor]['Inter'] += 1/size_actors

                if actor in eu:
                    dic_countries_frac['EU']['Inter'] += 1/size_actors


In [22]:
"""df_country = pd.DataFrame.from_dict(data=dic_countries, orient="index", columns=['SDG', 'DT', 'SDG-DT'])
df_country_eu = df_country.loc[eu_countries, :]
df_country_eu.index.names = ['Country']

df_country_eu = df_country_eu.merge(gdppop, left_index=True, right_index=True)
df_country_eu.loc[:, 'SDG-pop'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'DT-pop'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'SDG-DT-pop'] = df_country_eu.loc[:, 'SDG-DT'] /  df_country_eu.loc[:, 'avg_pop']

df_country_eu.loc[:, 'SDG-gdp'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'DT-gdp'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'SDG-DT-gdp'] = df_country_eu.loc[:, 'SDG-DT'] /  df_country_eu.loc[:, 'gdp']

df_country_eu"""

df_country = pd.DataFrame.from_dict(data=dic_countries_frac, orient="index")
df_country_eu = df_country.loc[eu, :]

df_country_eu.index.names = ['Country']

df_country_eu = df_country_eu.merge(gdppop, left_index=True, right_index=True)
df_country_eu.loc[:, 'SDG-pop'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'DT-pop'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'avg_pop']
df_country_eu.loc[:, 'Inter-pop'] = df_country_eu.loc[:, 'Inter'] /  df_country_eu.loc[:, 'avg_pop']

df_country_eu.loc[:, 'SDG-gdp'] = df_country_eu.loc[:, 'SDG'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'DT-gdp'] = df_country_eu.loc[:, 'DT'] /  df_country_eu.loc[:, 'gdp']
df_country_eu.loc[:, 'Inter-gdp'] = df_country_eu.loc[:, 'Inter'] /  df_country_eu.loc[:, 'gdp']
#
df_country_eu

,DT,SDG,Inter,avg_pop,gdp,SDG-pop,DT-pop,Inter-pop,SDG-gdp,DT-gdp,Inter-gdp
Austria,3236.053303,5372.635315,138.258313,8.665144e+06,92140.6,0.000620,0.000373,0.000016,0.058309,0.035121,0.001501
Belgium,4779.746296,8371.507113,181.901241,1.128651e+07,118204.0,0.000742,0.000423,0.000016,0.070823,0.040436,0.001539
Bulgaria,549.483471,1009.169897,24.554932,7.146214e+06,14512.3,0.000141,0.000077,0.000003,0.069539,0.037863,0.001692
Croatia,1225.321102,2621.667351,78.026927,4.161662e+06,12658.3,0.000630,0.000294,0.000019,0.207111,0.096800,0.006164
Cyprus,541.687904,851.738882,41.854211,1.165859e+06,5570.4,0.000731,0.000465,0.000036,0.152904,0.097244,0.007514
Czech Republic,2647.521995,4618.636023,107.792072,1.057757e+07,53083.1,0.000437,0.000250,0.000010,0.087008,0.049875,0.002031
Denmark,3078.092221,7977.961969,143.020285,5.703408e+06,77598.3,0.001399,0.000540,0.000025,0.102811,0.039667,0.001843
Estonia,438.769553,1238.384147,31.906383,1.322525e+06,6751.5,0.000936,0.000332,0.000024,0.183424,0.064988,0.004726
Finland,3615.406532,7719.220726,200.267440,5.471458e+06,57830.0,0.001411,0.000661,0.000037,0.133481,0.062518,0.003463
France,15853.700722,23290.960538,551.407769,6.648011e+07,604440.4,0.000350,0.000238,0.000008,0.038533,0.026229,0.000912


In [23]:
# df_country_eu.loc['max']= df_country_eu.max()
# df_country_eu

In [24]:
df_country_eu.columns

Index(['DT', 'SDG', 'Inter', 'avg_pop', 'gdp', 'SDG-pop', 'DT-pop',
       'Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp'],
      dtype='object')

In [25]:
for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
    tit = cat.replace("-pop", "").replace("-gdp", "")
    print(tit)

SDG
DT
Inter
SDG
DT
Inter
SDG
DT
Inter


## Maps for EU

In [25]:
for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
    # tit = cat.replace("-pop", "").replace("-gdp", "")
    if 'pop' in cat:
        spec = "/pop*"
        annot= "*Normalised by population"
    elif "gdp" in cat:
        spec = "/gdp*"
        annot = "*Normalised by GDP (in M€)"
    else:
        spec = ""
        annot = ""
    tit = f"Nb pubs{spec}"
    layout = go.Layout(
        annotations=[
            dict(
                x=1.05,
                y=1,
                align="right",
                valign="top",
                text=tit,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=22,
                ),
            dict(
                x=0.25,
                y=-.01,
                align="right",
                valign="top",
                text=annot,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=30,
                ),
            ],
        )
    fig = go.Figure(
            data=go.Choropleth(
                locations=df_country_eu.index,
                z=df_country_eu[cat],
                text=df_country_eu.index,
                locationmode="country names",
                colorscale='Blues',
                autocolorscale=False,
                colorbar=dict(
                    len=0.9,
                    y=0.5
                )
            ),
            layout=layout,
        )

    fig.update_layout(
        template='simple_white',
        font=dict(size=30),
        showlegend=True,
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
            showframe=False
        ),
        height=1080,
        width=1200

    )
    fig.write_image(f"../img/Commission/maps/EU_{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

## Same on WORLD

In [26]:
%%time
set_countries = set()
for row in df_sdg.itertuples(index=False, name=None):
    cn = row[12]
    lst_actors = cn.split(", ")
    for actor in lst_actors:
        set_countries.add(actor)
set_countries.add('EU')

CPU times: user 3.53 s, sys: 57 µs, total: 3.53 s
Wall time: 3.53 s


In [27]:
len(set_countries)

218

In [28]:
# Digital

dic_countries = {country: [0, 0, 0]  for country in set_countries}

for row in df_digital.itertuples(index=False, name=None):
    lst_actors = row[idx_dt['CN']].split(", ")
    year = row[idx_dt['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        # Increment for digital
        if actor in dic_countries:
            dic_countries[actor][0] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][0] += 1 / size_actors

# SDG & SDG-DT
for row in df_sdg.itertuples(index=False, name=None):
    lst_actors = row[idx_sdg['CN']].split(", ")
    year = row[idx_sdg['PY']]
    size_actors = len(lst_actors)
    for actor in lst_actors:
        if actor in dic_countries:
            # Increment for SDG
            dic_countries[actor][1] += 1 / size_actors
            if actor in eu_countries:
                dic_countries['EU'][1] += 1 / size_actors
            if row[-1]:
                # Increment for SDG-DT
                dic_countries[actor][2] += 1 / size_actors
                if actor in eu_countries:
                    dic_countries['EU'][2] += 1 / size_actors


In [29]:
len(dic_countries)

218

In [30]:
dic_countries_to_rename = {"Palestine": "Palestinian Territory",
                                   "Cote d'Ivoire": "Ivory Coast",
                                   "Congo[DRC]": "Democratic Republic of the Congo",
                                   "Congo, Dem.Rep.": "Democratic Republic of the Congo",
                                   "Eswatini": "Swaziland",
                                   "Guinea Bissau": "Guinea-Bissau",
                                   "Timor-Leste": "East Timor",
                                   }
for country in dic_countries.copy():
    if country in dic_countries_to_rename:
        dic_countries[dic_countries_to_rename[country]] = dic_countries[country]

,ranking,gdppc,pop2022,gdp
Country,,,,
Luxembourg,1,118001,647.599,7.641733e+07
Singapore,2,97057,5975.689,5.799824e+08
Ireland,3,94392,5023.109,4.741413e+08
Qatar,4,93508,2695.122,2.520155e+08
Switzerland,5,72874,8740.472,6.369532e+08
...,...,...,...,...
Malawi,190,993,20405.317,2.026248e+07
Central African Republic,191,979,5579.144,5.461982e+06
Somalia,192,925,17597.511,1.627770e+07


In [39]:
df_country = pd.DataFrame.from_dict(data=dic_countries, orient="index", columns=['SDG', 'DT', 'Inter'])

df_country.index.names = ['Country']
dic_countries_to_rename = {"Palestine": "Palestinian Territory",
                                   "Cote d'Ivoire": "Ivory Coast",
                                   "Congo [DRC]": "Democratic Republic of the Congo",
                                   "Congo, Dem.Rep.": "Democratic Republic of the Congo",
                                   "Eswatini": "Swaziland",
                                   "Guinea Bissau": "Guinea-Bissau",
                                   "Timor-Leste": "East Timor",
                                   }
df_country.rename(index=dic_countries_to_rename, inplace=True)
df_country

,SDG,DT,Inter
Country,,,
United States,114589.956220,193487.922934,4689.826355
Czech Republic,2647.521995,4618.636023,107.792072
Kiribati,0.000000,6.486967,0.000000
Seychelles,0.055556,55.977470,0.055556
Swaziland,0.000000,32.625238,1.857143
...,...,...,...
Finland,3615.406532,7719.220726,200.267440
East Timor,1.000000,17.311742,0.500000
Ivory Coast,31.382757,242.738054,3.795635


In [40]:
df_country = df_country.merge(gdppop, left_index=True, right_index=True)
df_country

,SDG,DT,Inter,avg_pop,gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5
...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3


In [38]:
# df_country = df_country.replace(":", np.NAN).dropna(subset="gdp")
# for country in eu:
#     df_country.loc[country, :] = df_country.loc['EU', :]
# df_country

,SDG,DT,Inter,avg_pop,gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5
...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3


In [42]:
# df_country.loc[:, 'SDG-pop'] = df_country.loc[:, 'SDG'] /  df_country.loc[:, 'avg_pop']
# df_country.loc[:, 'DT-pop'] = df_country.loc[:, 'DT'] /  df_country.loc[:, 'avg_pop']
# df_country.loc[:, 'Inter-pop'] = df_country.loc[:, 'Inter'] /  df_country.loc[:, 'avg_pop']

df_country.loc[:, 'SDG-gdp'] = df_country.loc[:, 'SDG'] /  df_country.loc[:, 'gdp']
df_country.loc[:, 'DT-gdp'] = df_country.loc[:, 'DT'] /  df_country.loc[:, 'gdp']
df_country.loc[:, 'Inter-gdp'] = df_country.loc[:, 'Inter'] /  df_country.loc[:, 'gdp']

df_country

,SDG,DT,Inter,avg_pop,gdp,SDG-gdp,DT-gdp,Inter-gdp
Albania,33.306126,249.054814,2.220635,2.875263e+06,3348.7,0.009946,0.074374,0.000663
Algeria,2206.074170,1425.219143,94.420238,4.020870e+07,0.0,inf,inf,inf
Argentina,1119.582110,4854.826979,40.743442,4.333679e+07,0.0,inf,inf,inf
Armenia,51.328493,96.112800,4.187472,2.924618e+06,0.0,inf,inf,inf
Australia,17903.566199,46270.843816,972.265841,2.400823e+07,324962.5,0.055094,0.142388,0.002992
...,...,...,...,...,...,...,...,...
Turkey,12098.228238,15803.667151,503.252824,7.901677e+07,156571.8,0.077270,0.100936,0.003214
Ukraine,1002.330534,2733.750840,69.576506,4.498976e+07,0.0,inf,inf,inf
United Kingdom,30787.366532,61205.919539,1455.698402,6.524557e+07,609979.5,0.050473,0.100341,0.002386
United States,114589.956220,193487.922934,4689.826355,3.213942e+08,4518929.3,0.025358,0.042817,0.001038


In [56]:
df_country.loc[:, ['SDG', 'DT', 'Inter']] = df_country.loc[:, ['SDG', 'DT', 'Inter']].round(0)
df_country.loc[:, ['SDG-gdp', 'DT-gdp', 'Inter-gdp']] = df_country.loc[:, ['SDG-gdp', 'DT-gdp', 'Inter-gdp']]*1000

In [57]:
df_country.loc[['EU']+eu_countries+other_countries, ['SDG', 'DT', 'Inter', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']].to_excel("frac_countries.xlsx")

In [74]:
# for cat in ['SDG', 'DT', 'Inter', 'SDG-pop', 'DT-pop','Inter-pop', 'SDG-gdp', 'DT-gdp', 'Inter-gdp']:
for cat in ['SDG', 'DT', 'Inter']:
    # tit = cat.replace("-pop", "").replace("-gdp", "")
    if 'pop' in cat:
        spec = "/pop*"
        annot= "*Normalised by population"
    elif "gdp" in cat:
        spec = "/gdp*"
        annot = "*Normalised by GDP (in M$)"
    else:
        spec = ""
        annot = ""
    tit = f"Nb pubs{spec}"

    layout = go.Layout(
        annotations=[
            dict(
                x=1,
                y=0.9,
                align="right",
                valign="top",
                text=tit,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=22,
                ),
            dict(
                x=0.25,
                y=-.01,
                align="right",
                valign="top",
                text=annot,
                showarrow=False,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                font_size=30,
                ),
            ],
        )
    fig = go.Figure(
            data=go.Choropleth(
                locations=df_country.index,
                z=df_country[cat],
                text=df_country.index,
                locationmode="country names",
                colorscale='Blues',
                autocolorscale=False,
                colorbar=dict(
                    len=0.8,
                    y=0.45,
                    x= 0.95
                )
            ),
            layout=layout,
        )

    fig.update_layout(
        template='simple_white',
        font=dict(size=25),
        showlegend=True,

        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=1,
            center=dict(lon=20, lat=30),
            projection_type="kavrayskiy7",

            showland=True,
            showcountries=False,
            showframe=False
        ),
        height=900,
        width=1400
    )

    fig.write_image(f"../img/Commission/maps/World/{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

## Specialization

In [59]:
df_country_w_continent = pd.read_excel("../data/countries_w_continent.xlsx", sheet_name=0, index_col=0)
df_country_w_continent

,Country,Country ISO3,Country Code,Long Name,Income Group,Lending Category,Region,Currency Unit,Other Groups,WTO Member,latitude,longitude
0,Afghanistan,AFG,4,Islamic State of Afghanistan,Low income,IDA,South Asia,Afghan afghani,HIPC,No,33.939110,67.709953
1,Albania,ALB,8,Republic of Albania,Upper middle income,IBRD,Europe & Central Asia,Albanian lek,NaN,Yes,41.153332,20.168331
2,Algeria,DZA,12,People's Democratic Republic of Algeria,Upper middle income,IBRD,Middle East & North Africa,Algerian dinar,NaN,No,28.033886,1.659626
3,American Samoa,ASM,16,American Samoa,Upper middle income,NaN,East Asia & Pacific,U.S. dollar,NaN,No,-14.270972,-170.132217
4,Andorra,AND,20,NaN,Others,NaN,Europe & Central Asia,NaN,NaN,No,42.546245,1.601554
...,...,...,...,...,...,...,...,...,...,...,...,...
265,Venezuela,VEN,862,República Bolivariana de Venezuela,High income: nonOECD,IBRD,Latin America & Caribbean,Venezuelan bolivar fuerte,NaN,Yes,6.423750,-66.589730
266,Vietnam,VNM,704,Socialist Republic of Vietnam,Lower middle income,Blend,East Asia & Pacific,Vietnamese dong,NaN,Yes,14.058324,108.277199
270,Yemen,YEM,887,Republic of Yemen,Lower middle income,IDA,Middle East & North Africa,Yemeni rial,NaN,No,15.552727,48.516388
273,Zambia,ZMB,894,Republic of Zambia,Lower middle income,IDA,Sub-Saharan Africa,New Zambian kwacha,HIPC,Yes,-13.133897,27.849332


In [61]:
lst_sdg = ["SDG" + str(i) for i in range(1, 18)]
lst_cat = ["Environment", "Society", "Economy"]
lst_dt_shortened = ['AI', 'robotics', 'IOT']
lst_dt = ['AI', 'big_data', 'IOT', 'computing_infrastructure', 'blockchain', 'robotics',
          'additive_manufacturing']
lst_cat_dt = ['Environment-AI', 'Environment-robotics', 'Environment-IOT',
              'Society-AI', 'Society-robotics', 'Society-IOT',
              'Economy-AI', 'Economy-robotics', 'Economy-IOT']
df_sdg = pd.read_pickle("../data/dataframes/output/all_sdg_fixed_dst.pkl")
df_dt = pd.read_pickle("../data/dataframes/DT/dt_updated.pkl")

# Filtering for Journal only
df_sdg = df_sdg[df_sdg.PT == 'J']
df_dt = df_dt[df_dt.PT == 'J']
df_sdg_dt = df_sdg[df_sdg['DST']]

In [62]:
%%time
import itertools
from tqdm import tqdm
dic = {c:{elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt} for c in set_countries}
dic_world = {elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt}

# SDG dataframe
for row in tqdm(df_sdg.itertuples(index=False, name=None), total=df_sdg.shape[0]):
    lst_actors = row[idx_sdg['CN']].split(", ")
    size_actors = len(lst_actors)
    sdgs_in_pubs = []
    # Country count
    for sdg in lst_sdg:
        if row[idx_sdg[sdg]]:
            sdgs_in_pubs += sdg
            for actor in lst_actors:
                dic_world[sdg] += 1
                dic[actor][sdg] += 1
    for cat in lst_cat:
        if row[idx_sdg[cat]]:
            for actor in lst_actors:
                dic_world[cat] += 1
                dic[actor][cat] += 1


# DT dataframe
for row in tqdm(df_dt.itertuples(index=False, name=None), total=df_dt.shape[0]):
    lst_actors = row[idx_dt['CN']].split(", ")
    # Country count
    for dt in lst_dt:
        if row[idx_dt[dt]]:
            for actor in lst_actors:
                if actor in dic:
                    dic[actor][dt] += 1
                    dic_world[dt] += 1

# Intersection
for row in tqdm(df_sdg_dt.itertuples(index=False, name=None), total=df_sdg_dt.shape[0]):
    # Country count
    lst_actors = row[idx_sdg_dt['CN']].split(", ")
    for cat in lst_cat:
        if row[idx_sdg_dt[cat]]:
            for dt in lst_dt:
                if row[idx_sdg_dt[dt]]:
                    for actor in lst_actors:
                        if str(cat) + "-" + str(dt) in lst_cat_dt:
                            dic_world[str(cat) + "-" + str(dt)] += 1
                            dic[actor][str(cat) + "-" + str(dt)] += 1


100%|██████████| 33097/33097 [00:00<00:00, 140127.67it/s]

CPU times: user 9.58 s, sys: 2.33 ms, total: 9.58 s
Wall time: 9.58 s


In [33]:
"""
%%time
# Fractional version

import itertools
from tqdm import tqdm
dic = {c:{elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt} for c in set_countries}
dic_world = {elem:0 for elem in lst_sdg+lst_cat+lst_dt+lst_cat_dt}

# SDG dataframe
for row in tqdm(df_sdg.itertuples(index=False, name=None), total=df_sdg.shape[0]):
    lst_actors = row[idx_sdg['CN']].split(", ")
    size_actors = len(lst_actors)
    sdgs_in_pub = []
    cats_in_pub = []
    # Country count
    for sdg in lst_sdg:
        if row[idx_sdg[sdg]]:
            sdgs_in_pub.append(sdg)
    for cat in lst_cat:
        if row[idx_sdg[cat]]:
            cats_in_pub.append(cat)
    tot_sdg_in_pub = len(sdgs_in_pub)
    tot_cat_in_pub = len(cats_in_pub)
    for sdg in sdgs_in_pub:
        dic_world[sdg] += 1/tot_sdg_in_pub
    for cat in cats_in_pub:
        dic_world[cat] += 1/tot_cat_in_pub

    for actor in lst_actors:
        for sdg in sdgs_in_pub:
            dic[actor][sdg] += 1/(size_actors*tot_sdg_in_pub)
        for cat in cats_in_pub:
            dic[actor][cat] += 1/(size_actors*tot_cat_in_pub)


# DT dataframe
for row in tqdm(df_dt.itertuples(index=False, name=None), total=df_dt.shape[0]):
    lst_actors = row[idx_dt['CN']].split(", ")
    tot_actors = len(lst_actors)
    dt_in_pub = []
    # Country count
    for dt in lst_dt:
        if row[idx_dt[dt]]:
            dt_in_pub.append(dt)
    tot_dt = len(dt_in_pub)
    for dt in dt_in_pub:
        dic_world[dt] += 1/tot_dt
    for actor in lst_actors:
        if actor in dic:
            for dt in dt_in_pub:
                dic[actor][dt] += 1/(tot_actors*tot_dt)


# Intersection
for row in tqdm(df_sdg_dt.itertuples(index=False, name=None), total=df_sdg_dt.shape[0]):
    # Country count
    lst_actors = row[idx_sdg_dt['CN']].split(", ")
    tot_actors = len(lst_actors)
    inter_in_pub = []
    for cat in lst_cat:
        if row[idx_sdg_dt[cat]]:
            for dt in lst_dt:
                if row[idx_sdg_dt[dt]]:
                    inter_in_pub.append(str(cat) + "-" + str(dt))
    tot_inter = len(inter_in_pub)
    for inter in inter_in_pub:
        if inter in dic_world:
            dic_world[inter] += 1/tot_inter

    for actor in lst_actors:
        for inter in inter_in_pub:
            if inter in lst_cat_dt:
                dic[actor][inter] += (1 / (tot_inter*tot_actors))
"""


100%|██████████| 33097/33097 [00:00<00:00, 163085.32it/s]

CPU times: user 10 s, sys: 6.83 ms, total: 10 s
Wall time: 10 s


In [63]:
df_spec_raw = pd.DataFrame.from_dict(data=dic, orient="index", columns=lst_sdg+lst_cat+lst_dt+lst_cat_dt)
df_spec_raw


,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,additive_manufacturing,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT
United States,35691,13659,160936,64511,59759,51527,25011,15220,12575,10377,...,48039,5836,2342,841,7546,1716,2268,1253,454,397
Czech Republic,204,241,1791,486,1173,3184,699,417,387,381,...,2086,122,59,19,155,58,37,32,12,17
Kiribati,6,7,0,1,4,3,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Seychelles,1,1,14,6,54,7,1,0,4,0,...,0,0,2,0,0,0,0,0,0,0
Eswatini,17,2,58,11,5,13,7,4,0,0,...,0,0,0,0,11,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brazil,3522,4358,19545,2880,8478,22590,4766,1314,2306,1615,...,4181,1015,287,70,1056,224,270,204,45,93
Canada,5492,2333,22182,8267,11843,14299,4583,2215,2310,3014,...,5998,1128,262,127,1345,208,422,200,38,91
Nigeria,1356,766,4571,760,944,1841,592,575,789,415,...,89,108,10,18,160,6,37,53,7,3
Somalia,15,8,43,3,8,6,0,4,1,1,...,0,0,0,0,2,0,0,2,0,0


In [64]:
df_spec_raw.loc['world', :] = df_spec_raw.sum(axis=0)
df_spec_raw

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,additive_manufacturing,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT
United States,35691.0,13659.0,160936.0,64511.0,59759.0,51527.0,25011.0,15220.0,12575.0,10377.0,...,48039.0,5836.0,2342.0,841.0,7546.0,1716.0,2268.0,1253.0,454.0,397.0
Czech Republic,204.0,241.0,1791.0,486.0,1173.0,3184.0,699.0,417.0,387.0,381.0,...,2086.0,122.0,59.0,19.0,155.0,58.0,37.0,32.0,12.0,17.0
Kiribati,6.0,7.0,0.0,1.0,4.0,3.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Seychelles,1.0,1.0,14.0,6.0,54.0,7.0,1.0,0.0,4.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Eswatini,17.0,2.0,58.0,11.0,5.0,13.0,7.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Canada,5492.0,2333.0,22182.0,8267.0,11843.0,14299.0,4583.0,2215.0,2310.0,3014.0,...,5998.0,1128.0,262.0,127.0,1345.0,208.0,422.0,200.0,38.0,91.0
Nigeria,1356.0,766.0,4571.0,760.0,944.0,1841.0,592.0,575.0,789.0,415.0,...,89.0,108.0,10.0,18.0,160.0,6.0,37.0,53.0,7.0,3.0
Somalia,15.0,8.0,43.0,3.0,8.0,6.0,0.0,4.0,1.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
Finland,638.0,388.0,3018.0,1858.0,2365.0,3850.0,1673.0,537.0,674.0,733.0,...,1574.0,220.0,94.0,59.0,225.0,51.0,133.0,36.0,22.0,75.0


In [65]:
df_world = pd.DataFrame.from_dict(data=dic_world, orient="index", columns=['tot']).transpose()
df_world

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,...,additive_manufacturing,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT
tot,120625,89733,706438,190477,300975,598317,225075,72622,93317,74840,...,264913,37686,11815,6307,50369,10432,20237,10094,2394,4320


In [66]:
# Keeping only some relevant countries
all_pub_wos = pd.read_excel("../data/total_number_pubs_2010_2022.xlsx", index_col=0)
tot_pub_countries = all_pub_wos.loc['TOT', :]
index_countries = list(tot_pub_countries.index)[2:]
lst_eu = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark",
          "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy",
          "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland",
          "Portugal", "Romania" ,"Slovakia", "Slovenia", "Spain", "Sweden"]
lst_eu.sort()
rest = sorted(list(set(index_countries).difference(set(lst_eu))))
rest.remove('Liechtenstein')
rest

['Australia',
 'Canada',
 'China',
 'Iceland',
 'Israel',
 'Japan',
 'Norway',
 'South Korea',
 'Switzerland',
 'United Kingdom',
 'United States']

In [67]:
df_spec_raw_categories = df_spec_raw.loc[:, ['Environment', 'Society', 'Economy']]
df_spec_raw_categories.loc[:, 'tot'] = df_spec_raw_categories.sum(axis=1)


In [68]:
df_spec_raw_dt = df_spec_raw.loc[:, lst_dt]
df_spec_raw_dt.loc[:, 'tot'] = df_spec_raw_dt.sum(axis=1)
df_spec_raw_dt

,AI,big_data,IOT,computing_infrastructure,blockchain,robotics,additive_manufacturing,tot
United States,325488.0,42387.0,66839.0,47419.0,3199.0,107248.0,48039.0,640619.0
Czech Republic,7026.0,547.0,1330.0,947.0,98.0,2245.0,2086.0,14279.0
Kiribati,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Seychelles,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
Eswatini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
Canada,51587.0,5481.0,11893.0,8177.0,901.0,15687.0,5998.0,99724.0
Nigeria,2368.0,236.0,406.0,408.0,50.0,255.0,89.0,3812.0
Somalia,5.0,0.0,1.0,2.0,0.0,1.0,0.0,9.0
Finland,10191.0,1282.0,3772.0,2740.0,274.0,2622.0,1574.0,22455.0


In [69]:
df_spec_raw_sdg = df_spec_raw.loc[:, lst_sdg]
df_spec_raw_sdg.loc[:, 'tot'] = df_spec_raw_sdg.sum(axis=1)
df_spec_raw_sdg

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15,SDG16,SDG17,tot
United States,35691.0,13659.0,160936.0,64511.0,59759.0,51527.0,25011.0,15220.0,12575.0,10377.0,31495.0,39900.0,237917.0,124120.0,58833.0,45203.0,29878.0,1016612.0
Czech Republic,204.0,241.0,1791.0,486.0,1173.0,3184.0,699.0,417.0,387.0,381.0,1161.0,1726.0,8637.0,2064.0,2994.0,732.0,630.0,26907.0
Kiribati,6.0,7.0,0.0,1.0,4.0,3.0,6.0,0.0,0.0,0.0,0.0,3.0,12.0,11.0,0.0,0.0,5.0,58.0
Seychelles,1.0,1.0,14.0,6.0,54.0,7.0,1.0,0.0,4.0,0.0,1.0,1.0,31.0,283.0,19.0,1.0,13.0,437.0
Eswatini,17.0,2.0,58.0,11.0,5.0,13.0,7.0,4.0,0.0,0.0,9.0,4.0,29.0,2.0,18.0,7.0,6.0,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Canada,5492.0,2333.0,22182.0,8267.0,11843.0,14299.0,4583.0,2215.0,2310.0,3014.0,5668.0,8629.0,52424.0,24875.0,11203.0,7787.0,5252.0,192376.0
Nigeria,1356.0,766.0,4571.0,760.0,944.0,1841.0,592.0,575.0,789.0,415.0,1048.0,1442.0,2841.0,1536.0,1835.0,1726.0,1678.0,24715.0
Somalia,15.0,8.0,43.0,3.0,8.0,6.0,0.0,4.0,1.0,1.0,7.0,5.0,17.0,3.0,11.0,11.0,9.0,152.0
Finland,638.0,388.0,3018.0,1858.0,2365.0,3850.0,1673.0,537.0,674.0,733.0,1818.0,2634.0,13520.0,4536.0,3598.0,1290.0,839.0,43969.0


In [70]:
df_spec_raw_sdg_dt = df_spec_raw.loc[:, lst_cat_dt]
df_spec_raw_sdg_dt.loc[:, 'tot'] = df_spec_raw_sdg_dt.sum(axis=1)
df_spec_raw_sdg_dt

,Environment-AI,Environment-robotics,Environment-IOT,Society-AI,Society-robotics,Society-IOT,Economy-AI,Economy-robotics,Economy-IOT,tot
United States,5836.0,2342.0,841.0,7546.0,1716.0,2268.0,1253.0,454.0,397.0,22653.0
Czech Republic,122.0,59.0,19.0,155.0,58.0,37.0,32.0,12.0,17.0,511.0
Kiribati,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Seychelles,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Eswatini,0.0,0.0,0.0,11.0,5.0,0.0,0.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...
Canada,1128.0,262.0,127.0,1345.0,208.0,422.0,200.0,38.0,91.0,3821.0
Nigeria,108.0,10.0,18.0,160.0,6.0,37.0,53.0,7.0,3.0,402.0
Somalia,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,4.0
Finland,220.0,94.0,59.0,225.0,51.0,133.0,36.0,22.0,75.0,915.0


In [48]:
lst_df_spec = []
for df_spec_raw in [df_spec_raw_categories,df_spec_raw_sdg, df_spec_raw_dt, df_spec_raw_sdg_dt]:
    df_spec = df_spec_raw.copy().iloc[:-1, :-1].loc[lst_eu+rest, :]
    for cat in df_spec.columns:
        for country in df_spec.index:
            df_spec.loc[country,cat] = (df_spec_raw.loc[country,cat] / df_spec_raw.loc[country, 'tot']) / (df_spec_raw.loc["world", cat] / df_spec_raw.loc['world', 'tot'])
    lst_df_spec.append(df_spec)


In [49]:
with pd.ExcelWriter('output.xlsx') as writer:
     for i, df in enumerate(lst_df_spec):
        df.to_excel(writer, sheet_name=f'Sheet{i+1}')

## New Specialization graphs

In [50]:
def transform_discrete_value(value):
    if value < 0.75:
        return "Under-specialized (RTA < 0.75)"
    elif 0.75 <= value <= 1.25:
        return "Not-specialized (0.75 <= RTA <= 1.25)"
    elif value > 1.25:
        return "Specialized (RTA > 1.25)"
    else:
        return "error"

In [51]:
def reorder_values(dataframe, col):
    """

    Args:
        dataframe:
        col:

    Returns:
        The dataframe reordered so that Specialized is on top, Not specialized in the middle, and under specialized is
        at the bottom of the legend
    """
    lst_index = list(dataframe.index)

    bottom_index = dataframe.loc[dataframe[col] == "Under-specialized (RTA < 0.75)"].index
    middle_index = dataframe.loc[dataframe[col] == "Not-specialized (0.75 <= RTA <= 1.25)"].index
    top_index = dataframe.loc[dataframe[col] == "Specialized (RTA > 1.25)"].index

    new_order = [top_index, middle_index, bottom_index]
    new_order_single_elem = []
    for elem in new_order:
        if len(elem) > 0:
            new_order_single_elem.append(elem[0])
            lst_index.remove(elem[0])

    return new_order_single_elem + lst_index



### CATGORIES SPEC GRAPHS

In [52]:
import plotly.express as px
# df_spec_cat = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="cat", index_col=0)
df_spec_cat = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="categories", index_col=0)
df_spec_cat = df_spec_cat.loc[lst_eu, :]
df_spec_cat = df_spec_cat.apply(np.vectorize(transform_discrete_value))


for cat in lst_cat:
    ordered_index = reorder_values(df_spec_cat, cat)
    fig = px.choropleth(
        # locations=df_spec_cat.loc[lst_eu, :].index,
        locations=ordered_index,
        color=df_spec_cat.loc[ordered_index, cat],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,
        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/{cat}.jpg")
    # pio.write_image(fig, folder, format='png')

### DT SPEC GRAPSH

In [53]:
import plotly.express as px
# df_spec_dt =pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="dt", index_col=0)
df_spec_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="DT", index_col=0)
df_spec_dt = df_spec_dt.loc[lst_eu, :]
df_spec_dt = df_spec_dt.apply(np.vectorize(transform_discrete_value))
for dt in ['AI', 'IOT', 'robotics']:
    ordered_index = reorder_values(df_spec_dt, dt)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_dt.loc[ordered_index, dt],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count//{dt}.jpg")
    # pio.write_image(fig, folder, format='png')

## CAT DT SPEC GRAPHS

In [75]:
df_spec_cat_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG-DT", index_col=0)

df_spec_cat_dt.loc[eu_countries+other_countries, ['Economy-AI', 'Economy-IOT', 'Environment-AI', 'Environment-IOT', 'Society-AI', 'Society-IOT']].to_excel("frac_countries_inter_cat.xlsx")

In [54]:
import plotly.express as px
# df_spec_cat_dt = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="cat-dt", index_col=0)
df_spec_cat_dt = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG-DT", index_col=0)

df_spec_cat_dt = df_spec_cat_dt.fillna(0)
df_spec_cat_dt = df_spec_cat_dt.loc[lst_eu, :]
df_spec_cat_dt = df_spec_cat_dt.apply(np.vectorize(transform_discrete_value))

for dt in df_spec_cat_dt.columns:
    ordered_index = reorder_values(df_spec_cat_dt, dt)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_cat_dt.loc[ordered_index, dt],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/{dt}.jpg")
    # pio.write_image(fig, folder, format='png')

## SDG SPEC GRAPHS

In [55]:
import plotly.express as px
# df_spec_sdg = pd.read_excel("../img/Commission/spec_fractional.xlsx", sheet_name="sdg", index_col=0)
df_spec_sdg = pd.read_excel("../img/Commission/spec.xlsx", sheet_name="SDG", index_col=0)
df_spec_sdg = df_spec_sdg.fillna(0)
df_spec_sdg = df_spec_sdg.loc[lst_eu, :]
df_spec_sdg = df_spec_sdg.apply(np.vectorize(transform_discrete_value))

for sdg in lst_sdg:
    ordered_index = reorder_values(df_spec_sdg, sdg)
    fig = px.choropleth(
        locations=ordered_index,
        color=df_spec_sdg.loc[ordered_index, sdg],
        color_discrete_map={"Under-specialized (RTA < 0.75)":"#2ECBE9", "Not-specialized (0.75 <= RTA <= 1.25)":"#128FC8", "Specialized (RTA > 1.25)":"#00468B"},
        locationmode="country names",
    )

    fig.update_layout(
        template='simple_white',
        font=dict(size=18),
        showlegend=True,

        legend=dict
            (
            title="Specialization",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0.048,
            bordercolor="Black",
            borderwidth=2
            ),
        geo=go.layout.Geo(
            scope="world",
            landcolor='lightgray',
            projection_scale=float(6.1),
            center=dict(lon=15, lat=52),
            projection_type="azimuthal equal area",
            showland=True,
            showcountries=False,
        ),
        height=1080,
        width=1200


    )
    fig.write_image(f"../img/Commission/maps/specialization/full_count/SDG/{sdg}.jpg")
    # pio.write_image(fig, folder, format='png')